In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import requests

# Polygon.io API key
API_KEY = 'ptaPp3UMqPfDic_ZNG929R8i2ohqXSuv'  # Replace with your actual API key

# Step 1: Download stock data from Polygon.io
def download_stock_data(ticker, days=1827):
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/2020-01-01/2024-10-01?apiKey={API_KEY}'
    response = requests.get(url)
    data = response.json()

    if 'results' in data:
        stock_data = pd.DataFrame(data['results'])
        stock_data['t'] = pd.to_datetime(stock_data['t'], unit='ms')  # Convert timestamp to datetime
        stock_data.set_index('t', inplace=True)
        stock_data['c'] = stock_data['c'].astype(float)  # Ensure close prices are floats
        return stock_data[['c']]  # Return only the closing prices
    else:
        raise Exception(f"Error fetching data: {data.get('message', 'Unknown error')}")

# Step 2: Preprocess the data
def preprocess_data(data, feature_col='c', seq_length=60):
    # Use 'Close' prices to predict trends
    data = data[[feature_col]]

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences of data points for LSTM input
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)

    # Reshape the data to be compatible with LSTM (samples, timesteps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler

# Step 3: Build the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Predicting a single output value (next price)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the LSTM model
def train_lstm_model(model, X_train, y_train, epochs=10, batch_size=64):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Make predictions and evaluate
def predict_and_evaluate(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)  # Rescale back to original values
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate the Root Mean Squared Error (RMSE)
    rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
    return predictions, rmse

# Step 6: Trading simulation logic to maximize profits
def simulate_trading(predictions, actual_prices, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []
    stop_loss_percentage = 0.05  # Stop-loss set at 5%

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price_today = actual_prices[i]
        actual_price_yesterday = actual_prices[i - 1]

        # Buy logic: Buy shares if today's actual price is higher than yesterday's price
        if actual_price_today > actual_price_yesterday and balance >= actual_price_today:
            shares_to_buy = balance // actual_price_today
            balance -= shares_to_buy * actual_price_today
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_price_today}, Balance: {balance}, Shares: {total_shares}")

        # Sell logic: Sell if the price has dropped by the stop-loss percentage
        if total_shares > 0 and (actual_price_today < (actual_price_today * (1 - stop_loss_percentage))):
            balance += total_shares * actual_price_today
            trade_log.append(f"Sold {total_shares} shares at {actual_price_today}, Balance: {balance}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1]}, Final Balance: {balance}")

    profit_loss = balance - initial_balance
    return trade_log, profit_loss

# Step 7: Full pipeline
def run_stock_prediction_with_simulation(ticker, seq_length=60):
    # Download and preprocess the data
    stock_data = download_stock_data(ticker)
    X, y, scaler = preprocess_data(stock_data, seq_length=seq_length)

    # Prepare data for training and testing
    dates = stock_data.index[seq_length:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], 1))
    model = train_lstm_model(model, X_train, y_train, epochs=10)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Simulate trading
    trade_log, profit_loss = simulate_trading(predictions.flatten(), actual_prices.flatten())

    print(f"Root Mean Squared Error: {rmse}")
    print(f"Final Profit/Loss: {profit_loss}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss

# Example usage
if __name__ == "__main__":
    ticker_symbol = 'GOOGL'  # Change to any stock ticker you want to test
    stock_data, predictions, trade_log, profit_loss = run_stock_prediction_with_simulation(ticker_symbol)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - loss: 0.1439
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0242
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 0.0118
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0125
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - loss: 0.0081
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0070
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 0.0064
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0052
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0046
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0052
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
Root Mean Squared Error: 6.565369630627709
Final Profit/Loss: -387.59999999999854
Bought 57.0 shares at 173.79, Balance: 93.97000000000116, Shares: 57.0
Final Sale of 57.0 shares at 166.99, Final Balance: 9612.400000000001
